In [1]:
import os
import pdb
import torch
import pandas
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv, GATv2Conv
import torch.nn.functional as F

In [2]:
dataset = Planetoid(root='data/Planetoid', name='Pubmed', transform=NormalizeFeatures())

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Dataset: Pubmed():
Number of graphs: 1
Number of features: 500
Number of classes: 3
Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
Number of nodes: 19717
Number of edges: 88648
Average node degree: 4.50
Number of training nodes: 60
Training node label rate: 0.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [3]:
class GATv2(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)
        
        self.conv1 = GATv2Conv(dataset.num_features, hidden_channels, dropout=0.6)
        self.conv2 = GATv2Conv(hidden_channels, dataset.num_classes, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [4]:
model = GATv2(hidden_channels=8, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def test(mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
    acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
    return acc

for epoch in range(1, 101):
    loss = train()
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

GATv2(
  (conv1): GATv2Conv(500, 8, heads=1)
  (conv2): GATv2Conv(8, 3, heads=1)
)
Epoch: 001, Loss: 1.1355, Val: 0.3880, Test: 0.4130
Epoch: 002, Loss: 1.1148, Val: 0.3880, Test: 0.4130
Epoch: 003, Loss: 1.1102, Val: 0.3880, Test: 0.4130
Epoch: 004, Loss: 1.1045, Val: 0.3880, Test: 0.4130
Epoch: 005, Loss: 1.1142, Val: 0.3880, Test: 0.4130
Epoch: 006, Loss: 1.0893, Val: 0.3880, Test: 0.4130
Epoch: 007, Loss: 1.0867, Val: 0.3880, Test: 0.4130
Epoch: 008, Loss: 1.0703, Val: 0.3880, Test: 0.4130
Epoch: 009, Loss: 1.0867, Val: 0.4100, Test: 0.4610
Epoch: 010, Loss: 1.0798, Val: 0.5760, Test: 0.5800
Epoch: 011, Loss: 1.0852, Val: 0.6660, Test: 0.6360
Epoch: 012, Loss: 1.0790, Val: 0.6040, Test: 0.5810
Epoch: 013, Loss: 1.0929, Val: 0.5140, Test: 0.4830
Epoch: 014, Loss: 1.0544, Val: 0.4540, Test: 0.4360
Epoch: 015, Loss: 1.1226, Val: 0.3900, Test: 0.3980
Epoch: 016, Loss: 1.0551, Val: 0.3580, Test: 0.3650
Epoch: 017, Loss: 1.0579, Val: 0.3360, Test: 0.3130
Epoch: 018, Loss: 1.0721, Val: 0.

In [5]:
test_acc = test(data.val_mask)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7400
